In [1]:
import pandas as pd
import tkinter as tk
import random
import re
import json
from tkinter import messagebox

In [2]:
VALIDATION_TYPE = 'ORACLE_HUMAN'
DATE = 160924
POLYMER_TYPE = 'OMIEC_NEUROMORPHIC'

FILE_RESPONSE_PATH = '../../dataframes/llama3-70B-OMIEC_NEUROMORPHIC030924_0.2_temp_dict2_RESPONSES.xlsx'
RAW_DATA_PATH = '../../data/raw/OMIEC_NEUROMORPHIC_20_08_24_v2.xlsx'

In [3]:
response = pd.read_excel(FILE_RESPONSE_PATH)
raw_data =  pd.read_excel(RAW_DATA_PATH)

In [4]:
response

,index,response
0,0,"{ 'Polymer Mentioned Name': 'polythiophene', '..."
1,1,"{ 'Polymer Mentioned Name': 'melanin', 'Polyme..."
2,2,{ 'Polymer Mentioned Name': 'poly(3-[2-[2-(2-m...
3,3,{}
4,4,NaN
...,...,...
118,118,{}
119,119,{}
120,120,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...
121,121,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...


In [5]:
response

,index,response
0,0,"{ 'Polymer Mentioned Name': 'polythiophene', '..."
1,1,"{ 'Polymer Mentioned Name': 'melanin', 'Polyme..."
2,2,{ 'Polymer Mentioned Name': 'poly(3-[2-[2-(2-m...
3,3,{}
4,4,NaN
...,...,...
118,118,{}
119,119,{}
120,120,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...
121,121,{ 'Polymer Mentioned Name': 'poly(3-hexylthiop...


In [6]:
nan_count = response.isna().sum()
print(nan_count)

index        0
response    11
dtype: int64


In [7]:
result = pd.concat([response, raw_data], axis=1)
result.drop('index', axis=1, inplace=True)

In [8]:
response = response.dropna()
responses_cleaned = response[response['response'] != '{}']
responses_cleaned = responses_cleaned[responses_cleaned['response'] != 'None']

In [9]:
def prepare_dict(df):
    
    all_dicts = []
    index_nones = []
    for index, row in df.iterrows():
        try:
            if "PEDOT: PSS" in row['response']:
               row['response'] = row['response'].replace("PEDOT: PSS", "PEDOT:PSS")
            
            if "'Doped?:' " in row['response']:
                dicts = row['response'].strip().replace("'Doped?:'", "'Doped?':").rstrip(';').split(';')
            else:
                dicts = row['response'].strip().rstrip(';').split(';')
                
            dicts = [d.strip().replace("'", '"') for d in dicts]
            json_data = f"[{','.join(dicts)}]"
            dictt = json.loads(json_data)
        except Exception as e:
            print(index, '\n', row['response'], '\n')
            #print(row['response'].replace("'Doped?:'", "'Doped?':"))
            
        for polymer in dictt:
            #polymer['Index'] = index 
            if "Doped?:" in polymer:
                polymer["Doped?"] = polymer.pop("Doped?:")

            if 'Polymer Chemical Abbreviation' not in polymer:
                key = list(polymer.keys())[0]
                polymer['Polymer Chemical Abbreviation'] = polymer[key]
                
            if 'Polymer Mentioned Name' not in polymer:
                polymer['Polymer Mentioned Name'] = list(polymer.keys())[0]
                
            new_dict = {index: polymer}
            
            all_dicts.append(new_dict)
    
    return all_dicts, index_nones



In [10]:

polymer_dicts, _ = prepare_dict(responses_cleaned)

polymers = []

for polymer in polymer_dicts:
    print(polymer[list(polymer.keys())[0]])
    polymers.append(polymer[list(polymer.keys())[0]]['Polymer Chemical Abbreviation'])


{'Polymer Mentioned Name': 'polythiophene', 'Polymer Chemical Abbreviation': 'PTh', 'Polymer Application': 'neuromorphic computing, healthcare, bioelectronic devices', 'Doped?': 'p-doped'}
{'Polymer Mentioned Name': 'naphthalene diimide-based polymer', 'Polymer Chemical Abbreviation': 'NDI', 'Polymer Application': 'neuromorphic computing, healthcare, bioelectronic devices', 'Doped?': 'n-doped'}
{'Polymer Mentioned Name': 'melanin', 'Polymer Chemical Abbreviation': 'Mel', 'Polymer Application': 'bioelectronics, neuromorphic computing, soft robotics, energy storage, organic electrochemical transistors, supercapacitors', 'Doped?': 'not doped'}
{'Polymer Mentioned Name': 'poly(3,4-ethylenedioxythiophene) polystyrene sulfonate', 'Polymer Chemical Abbreviation': 'PEDOT:PSS', 'Polymer Application': 'organic electrochemical transistors, supercapacitors', 'Doped?': 'not doped'}
{'Polymer Mentioned Name': 'poly(3-[2-[2-(2-methoxyethoxy)ethoxy]ethyl]thiophene -2,5-diyl)', 'Polymer Chemical Abbrev

In [11]:
# Função para encontrar a interseção entre dois conjuntos de polímeros
def verificar_interseccao(p1, p2):
    conjunto1 = {p1['Polymer Chemical Abbreviation'].replace('/', '-').replace(':', '-').lower(),
                 p1['Polymer Mentioned Name'].replace('/', '-').replace(':', '-').lower()}
    conjunto2 = {p2['Polymer Chemical Abbreviation'].replace('/', '-').replace(':', '-').lower(),
                 p2['Polymer Mentioned Name'].replace('/', '-').replace(':', '-').lower()}
    return not conjunto1.isdisjoint(conjunto2)

# Agrupar polímeros com base na interseção
def agrupar_polimeros(polimeros):
    grupos = []
    visitados = set()
    
    for i, dados1 in enumerate(polimeros):
        
        if i in visitados:
            continue
        
        grupo_atual = set()
        grupo_nome_atual = []
        
        # Adicionar o polímero atual ao grupo
        grupo_atual.add(i)
        grupo_nome_atual.append([dados1[list(dados1.keys())[0]]['Polymer Chemical Abbreviation'], list(dados1.keys())[0]])
        visitados.add(i)
        
        # Comparar com elementos subsequentes
        for j in range(i + 1, len(polimeros)):
            dados2 = polimeros[j]
            if verificar_interseccao(dados1[list(dados1.keys())[0]], dados2[list(dados2.keys())[0]]):
                grupo_atual.add(j)
                grupo_nome_atual.append([dados2[list(dados2.keys())[0]]['Polymer Chemical Abbreviation'], list(dados2.keys())[0]])
                visitados.add(j)
        
        grupos.append(grupo_nome_atual)
    
    return grupos


grupos_nomes_encontrados = agrupar_polimeros(polymer_dicts)
#grupos_nomes_encontrados = sorted(grupos_nomes_encontrados, key=len, reverse=True)
grupos_nomes = [list(map(list, set(map(tuple, grupo)))) for _, grupo in enumerate(grupos_nomes_encontrados, start=1)]
grupos_nomes = sorted(grupos_nomes, key=len, reverse=True)
soma = 0

for i, grupo in enumerate(grupos_nomes, start=1):
    soma += len(grupo)
    #print(f"{grupo[0][0]}: {len(grupo)}")
print(f"Quantidade polímeros: {soma} | Quantidade polímeros únicos: {len(grupos_nomes)}")


Quantidade polímeros: 86 | Quantidade polímeros únicos: 51


In [12]:
items_to_remove = [r'(?i)\bmixed\b',
                   r'(?i)\bnone\b',
                   r'MIEC.*',
                   'based', 
                   r'[Pp]olymer',
                   r'OECT.*',
                   'conjug',
                   r'^$', 
                   r'^PSS$'
                   ]


In [13]:

print("Polímeros identificados antes:", len(grupos_nomes))
for item in items_to_remove:
    for p in grupos_nomes:
        pattern = re.compile(item)
        if pattern.search(p[0][0]):
            #print(item, '\n', p)
            grupos_nomes.remove(p)

print("Polímeros identificados após remoção:",len(grupos_nomes))

Polímeros identificados antes: 51
Polímeros identificados após remoção: 49


In [14]:
df = []
for i,grupo in enumerate(grupos_nomes, start=1):
    linha_df = {'Nome': grupo[0][0], 'Quantidade_Encontrada': len(grupo), 'Identificação_Encontrados': grupo}
    df.append(linha_df)
polímeros_estruturados = pd.DataFrame(df)
polímeros_estruturados

,Nome,Quantidade_Encontrada,Identificação_Encontrados
0,PEDOT:PSS,17,"[[PEDOT:PSS, 49], [PEDOT:PSS, 94], [PEDOT:PSS,..."
1,P3HT,6,"[[P3HT, 22], [P3HT, 121], [P3HT, 66], [P3HT, 4..."
2,PEDOT,5,"[[PEDOT, 10], [PEDOT, 41], [PEDOT, 44], [PEDOT..."
3,PTh,2,"[[PTh, 33], [PTh, 0]]"
4,BBL,2,"[[BBL, 102], [BBL, 12]]"
5,p(g2T-TT),2,"[[p(g2T-TT), 40], [p(g2T-TT), 41]]"
6,PPY,2,"[[PPY, 45], [PPy, 68]]"
7,p(g2T-T),2,"[[p(g2T-T), 50], [p(g2T-T), 57]]"
8,CPE-K,2,"[[CPE-K, 107], [PCPDTBT-SO3K, 83]]"
9,PSSNa,2,"[[PSSNa, 109], [PSSNa, 85]]"


### Métrica  (Oracle)

In [15]:
random.seed(1024)
quantidade = int(len(polymer_dicts) * 0.20)
amostragem = random.sample(polymer_dicts, quantidade)
quantidade

17

In [16]:
# Dados de exemplo
responses_raw = {'Polímero': [], 'Abstract': [], 'Válido?': []}
c = 0
verificados = []

In [17]:
# Inicializando variáveis globais necessárias
c = 0

# Função para avançar para o próximo polímero
def next_sample(resp=None):  # Permitir None como valor padrão
    global c
    if resp is not None:  # Verifica se resp foi passado (evita adicionar a primeira chamada com string vazia)
        responses_raw['Válido?'].append(resp)  # Salva a resposta do usuário
    
    if c >= len(amostragem):
        messagebox.showinfo("Fim", "Todas as amostras foram verificadas!")
        root.destroy()  # Use destroy() para fechar a janela corretamente
        return

    # Pega a amostra atual com base no valor de c
    sample = amostragem[c]
    polymer = sample[list(sample.keys())[0]]['Polymer Mentioned Name']
    abstract = raw_data.loc[list(sample.keys())[0]]['Abstract']

    # Atualizando os valores nas listas responses_raw
    responses_raw['Polímero'].append(polymer)
    responses_raw['Abstract'].append(abstract)

    verificados.append(sample)

    # Atualizar os valores na interface do polímero
    polymer_text.config(state="normal")  # Tornar o widget editável para atualizar o conteúdo
    polymer_text.delete(1.0, tk.END)  # Remover o conteúdo anterior
    polymer_text.insert(tk.END, f"Polímero: {polymer}\n")  # Inserir o novo conteúdo
    polymer_text.tag_add("red", "1.9", "1.end")  # Aplicar a tag "red" ao polímero
    polymer_text.config(state="disabled")  # Desabilitar a edição novamente

    # Atualizar os valores na interface do abstract
    abstract_text.config(state="normal")
    abstract_text.delete(1.0, tk.END)
    abstract_text.insert(tk.END, f"Abstract: {abstract}")

    # Procurar a posição do nome do polímero no abstract e aplicar a tag vermelha
    start_idx = abstract_text.search(polymer, "1.0", tk.END)  # Encontrar onde o polímero é mencionado no abstract
    if start_idx:
        end_idx = f"{start_idx}+{len(polymer)}c"  # Calcular o índice final com base no comprimento do nome do polímero
        abstract_text.tag_add("red", start_idx, end_idx)  # Aplicar a tag "red"
    
    abstract_text.config(state="disabled")
    
    # Incrementar o contador c para avançar para a próxima amostra
    c += 1

# Função para tratar o botão "True"
def on_true():
    next_sample("True")

# Função para tratar o botão "False"
def on_false():
    next_sample("False")

# Configurando a interface gráfica
root = tk.Tk()
root.title("Verificação de Polímeros")

# Frame para o polímero com barra de rolagem
polymer_frame = tk.Frame(root)
polymer_frame.pack(pady=10)

polymer_scroll = tk.Scrollbar(polymer_frame)
polymer_scroll.pack(side="right", fill="y")

polymer_text = tk.Text(polymer_frame, wrap="word", height=5, width=60, yscrollcommand=polymer_scroll.set)
polymer_text.pack()
polymer_scroll.config(command=polymer_text.yview)
polymer_text.tag_configure("red", foreground="red")  # Definir a tag "red" para destacar o polímero em vermelho
polymer_text.config(state="disabled")  # Desabilitar edição, apenas permitir seleção

# Frame para o abstract com barra de rolagem
abstract_frame = tk.Frame(root)
abstract_frame.pack(pady=10)

abstract_scroll = tk.Scrollbar(abstract_frame)
abstract_scroll.pack(side="right", fill="y")

abstract_text = tk.Text(abstract_frame, wrap="word", height=15, width=60, yscrollcommand=abstract_scroll.set)
abstract_text.pack()
abstract_scroll.config(command=abstract_text.yview)
abstract_text.tag_configure("red", foreground="red")  # Definir a tag "red" para o polímero no abstract
abstract_text.config(state="disabled")

# Botões para responder "True" ou "False"
btn_true = tk.Button(root, text="True", command=on_true, width=10)
btn_true.pack(side="left", padx=20, pady=20)

btn_false = tk.Button(root, text="False", command=on_false, width=10)
btn_false.pack(side="right", padx=20, pady=20)

# Inicializando o primeiro exemplo, sem passar resposta
next_sample()

# Iniciar a interface gráfica
root.mainloop()

In [18]:
df_oracle = pd.DataFrame(data=responses_raw)
model_accuracy = len(df_oracle[df_oracle['Válido?'] == 'True'])/len(df_oracle['Válido?'])
print(f'taxa de acerto do modelo: {model_accuracy}')

with open (f"precisao_modelo_{DATE}_{POLYMER_TYPE}.txt","w") as f:
    f.write(f"Data: {DATE} \n")
    f.write(f"Arquivos: {FILE_RESPONSE_PATH, RAW_DATA_PATH} \n")
    f.write(f"Taxa de acerto do modelo: {model_accuracy} \n")

taxa de acerto do modelo: 0.6470588235294118


In [19]:
# export
df_oracle.to_excel(f'llama3-70B-{POLYMER_TYPE}{DATE}_0.2_temp_dict_{VALIDATION_TYPE}.xlsx', index=False)

### Playground